In [2]:
import pandas as pd

In [3]:
pd.set_option("max_columns",1000)
pd.set_option("max_colwidth",1000)

# Load data 

In [4]:
tweets = pd.read_pickle("../../data/cp_tweets.pkl")

# Join with shape files

**Get a US states geojson file**    
Sources
- ArcGIS shapefile of US 50 states + DC (https://www.arcgis.com/home/item.html?id=f7f805eb65eb4ab787a0a3e1116ca7e5)


At the state level


In [7]:
import geopandas as gpd

In [8]:
states = gpd.read_file("../../data/states_21basic/states.shp")

In [50]:
states[["STATE_ABBR","STATE_NAME"]].head(10)

,STATE_ABBR,STATE_NAME
0,HI,Hawaii
1,WA,Washington
2,MT,Montana
3,ME,Maine
4,ND,North Dakota
5,SD,South Dakota
6,WY,Wyoming
7,WI,Wisconsin
8,ID,Idaho
9,VT,Vermont


In [32]:
import numpy as np

logic = (tweets.country == "United States") & \
        (tweets.place_place_type != "poi") & \
        (tweets.place_place_type != "country") & \
        (tweets.place_place_type != "neighborhood")\

grouped = tweets[logic][["place_full_name","place_place_type"]]\
.groupby(by=["place_place_type"]).count()

In [33]:
grouped

,place_full_name
place_place_type,
admin,150547
city,436470


In [84]:
states_cities = tweets[logic][["place_full_name","place_place_type"]].drop_duplicates()\
.sort_values("place_place_type",ascending=False)

In [36]:
logic1 = logic & (tweets.place_place_type == "admin")

tweets[logic1][["place_full_name","place_place_type"]].drop_duplicates()\
.sort_values("place_full_name")\
.head()

,place_full_name,place_place_type
100115,"Alabama, USA",admin
100268,"Alaska, USA",admin
10108,"Arizona, USA",admin
100008,"Arkansas, USA",admin
108890,Atlanta GA,admin


In [103]:
def get_state_ABRV(place_full_name):
    regex = r'([A-Z]){2,3}'
    match_ABRV = re.search(regex, place_full_name)
    
    if match_ABRV == None:
        print("could not find: ",place_full_name)
        return place_full_name
    
    abrv = match_ABRV.group()
    
    
    if abrv == "USA":
        regex = r'([A-Z])[a-z]*(?=,)'
        match_state_name = re.search(regex, place_full_name)
        
        if match_state_name == None:
            print("could not find: ",place_full_name)
            return place_full_name
        
        state_name = match_state_name.group()
        abrv = state_name_to_abrv(state_name)
        
    return abrv

In [106]:
state_name_pairs = states[["STATE_ABBR","STATE_NAME"]]\
.set_index("STATE_NAME")\
.to_dict().get("STATE_ABBR")


In [111]:
print(state_name_pairs)

{'Hawaii': 'HI', 'Washington': 'WA', 'Montana': 'MT', 'Maine': 'ME', 'North Dakota': 'ND', 'South Dakota': 'SD', 'Wyoming': 'WY', 'Wisconsin': 'WI', 'Idaho': 'ID', 'Vermont': 'VT', 'Minnesota': 'MN', 'Oregon': 'OR', 'New Hampshire': 'NH', 'Iowa': 'IA', 'Massachusetts': 'MA', 'Nebraska': 'NE', 'New York': 'NY', 'Pennsylvania': 'PA', 'Connecticut': 'CT', 'Rhode Island': 'RI', 'New Jersey': 'NJ', 'Indiana': 'IN', 'Nevada': 'NV', 'Utah': 'UT', 'California': 'CA', 'Ohio': 'OH', 'Illinois': 'IL', 'District of Columbia': 'DC', 'Delaware': 'DE', 'West Virginia': 'WV', 'Maryland': 'MD', 'Colorado': 'CO', 'Kentucky': 'KY', 'Kansas': 'KS', 'Virginia': 'VA', 'Missouri': 'MO', 'Arizona': 'AZ', 'Oklahoma': 'OK', 'North Carolina': 'NC', 'Tennessee': 'TN', 'Texas': 'TX', 'New Mexico': 'NM', 'Alabama': 'AL', 'Mississippi': 'MS', 'Georgia': 'GA', 'South Carolina': 'SC', 'Arkansas': 'AR', 'Louisiana': 'LA', 'Florida': 'FL', 'Michigan': 'MI', 'Alaska': 'AK'}


In [108]:
import pickle

a = state_name_pairs

with open('../../data/state_pairs.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../../data/state_pairs.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(a == b)

state_name_pairs = b

True


In [109]:
def state_name_to_abrv(state_name):
    abrv = state_name_pairs.get(state_name)
    return abrv

In [110]:
states_cities.place_full_name.apply(get_state_ABRV)

could not find:  Ceiba Baja, Puerto Rico
could not find:  St. Just, Puerto Rico
could not find:  Turabo, Puerto Rico
could not find:  Rincón, Puerto Rico
could not find:  Guaynabo, Puerto Rico
could not find:  Guaniquilla, Puerto Rico
could not find:  Cañabón, Puerto Rico
could not find:  Cruz Bay, Virgin Islands
could not find:  Guaraguao, Puerto Rico
could not find:  Hato Rey Central, Puerto Rico
could not find:  Vega Baja, Puerto Rico
could not find:  Añasco Abajo, Puerto Rico
could not find:  Caracol, Puerto Rico
could not find:  Perchas 2, Puerto Rico
could not find:  Toa Alta, Puerto Rico
could not find:  Cerro Gordo, Puerto Rico
could not find:  Santana, Puerto Rico
could not find:  Martín González, Puerto Rico
could not find:  Mata de Plátano, Puerto Rico
could not find:  Frederiksted, Virgin Islands
could not find:  Camarones, Puerto Rico
could not find:  Sabana Llana Norte, Puerto Rico
could not find:  Orocovis, Puerto Rico
could not find:  Salto Abajo, Puerto Rico
could not 

0           MO
238656      PA
240353      NJ
240314      FL
240243      CA
239958      NC
239891      KY
239863      TX
239778      SC
23954       IL
23950       NJ
239446      FL
239347      CA
23930       OH
238946      AR
238939      CA
23861       IL
240397      TX
238576      FL
238519      NY
238239      UT
238158      MS
237870      NC
23779       CA
23778       NJ
237567      PA
237537      IL
237475      NJ
237459      OH
23743       NY
          ... 
4392        AL
100115      AL
418842      IN
100062      TX
164852      VA
100063      IL
159178    None
100065      NE
104703      UT
100069      MD
100191      CA
100609      VT
100606      MI
60638       MN
193268      NY
100268      AK
167438      NY
10011       LA
105930      NY
100574      ID
103519    None
39529       TX
204366      CT
100093      TN
223830      MO
100100      CT
132774      NC
100548      VA
227822      ME
100120      ME
Name: place_full_name, Length: 8636, dtype: object

# Example Usage

In [142]:
tweets = pd.read_pickle("../../data/cp_tweets.pkl")

In [147]:
logic = (tweets.country == "United States") & \
        (tweets.place_place_type != "poi") & \
        (tweets.place_place_type != "country") & \
        (tweets.place_place_type != "neighborhood")

In [148]:
tweets_USD = tweets[logic]

In [149]:
# add helper lib to kernal running
import os
import sys

lib_dir = "../../libs/"
if lib_dir not in sys.path:
    sys.path.append(lib_dir)

In [150]:
# import helper lib
import DSF_helpers

In [153]:
tweets_USD["state_abrv"] = tweets_USD.place_full_name.apply(DSF_helpers.get_state_ABRV)

could not find:  Pueblo Viejo, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  Caguas, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  Mayagüez, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  Pueblo Viejo, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  Arecibo, Puerto Rico
could not find:  Charlotte Amalie, Virgin Islands
could not find:  El Cinco, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  Arecibo, Puerto Rico
could not find:  Navarro, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  El Cinco, Puerto Rico
could not find:  Arecibo, Puerto Rico
could not find:  Arecibo, Puerto Rico
could not find:  Arecibo, Puerto Rico
could not find

/Users/jeroen/anaconda3/envs/uva-DS-fundementals/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
tweets_USD[["place_full_name","state_abrv"]].head(100)

,place_full_name,state_abrv
0,"Frontenac, MO",MO
1,"Baton Rouge, LA",LA
10,"Chesapeake Beach, MD",MD
100,"Pennsylvania, USA",PA
1000,"Flemington, GA",GA
10000,"West Conshohocken, PA",PA
100000,"Erie, PA",PA
100001,"Ohio, USA",OH
100002,"Yuba City, CA",CA
100003,"Georgia, USA",GA


# Plot data